PC 환경

In [3]:
# @title 라이브러리 사용 정리

import time
import json
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2


In [2]:
# @title 로컬 경로 설정 및 JSON 파일 경로 수집

# Mac 로컬 환경의 데이터 기본 경로 설정
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
train_annotations_dir = os.path.join(base_data_dir, 'train_annotations') # train_annotations 폴더 경로

json_file_paths = [] # 모든 JSON 파일의 경로를 저장할 리스트

print(f"로컬 데이터 경로 설정 완료: {base_data_dir}")

start_time = time.time()

if os.path.exists(train_annotations_dir):
    for root, dirs, files in os.walk(train_annotations_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                full_path = os.path.join(root, file_name)
                json_file_paths.append(full_path)

    end_time = time.time()

    print(f"'train_annotations'에서 발견된 총 JSON 파일 수: {len(json_file_paths)}개")
    print(f"경로 수집 완료 시간: {end_time - start_time:.2f}초")
    print(f"첫 번째 JSON 파일 경로 (예시): {json_file_paths[0]}")

else:
    print(f"로컬 경로를 찾을 수 없습니다. {train_annotations_dir}")

로컬 데이터 경로 설정 완료: /Users/bellboi/data/ai05-level1-project
'train_annotations'에서 발견된 총 JSON 파일 수: 4526개
경로 수집 완료 시간: 0.29초
첫 번째 JSON 파일 경로 (예시): /Users/bellboi/data/ai05-level1-project/train_annotations/K-003483-025367-027777-035206_json/K-027777/K-003483-025367-027777-035206_0_2_0_2_75_000_200.json


In [5]:
!pip install ultralytics opencv-python

  Using cached ultralytics-8.3.220-py3-none-any.whl.metadata (37 kB)
  Using cached matplotlib-3.10.7-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached torch-2.9.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached torchvision-0.24.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (5.9 kB)
  Using cached ultralytics_thop-2.0.17-py3-none-any.whl.metadata (14 kB)
Using cached ultralytics-8.3.220-py3-none-any.whl (1.1 MB)
Using cached matplotlib-3.10.7-cp313-cp313-macosx_11_0_arm64.whl (8.1 MB)
Using cached torch-2.9.0-cp313-none-macosx_11_0_arm64.whl (74.5 MB)
Using cached torchvision-0.24.0-cp313-cp313-macosx_12_0_arm64.whl (1.9 MB)
Using cached ultralytics_thop-2.0.17-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [ultralytics] [ultralytics]


In [8]:
q# @title JSON 경로에서 이미지 추출 및 YOLOv5 추론
import os
import json
import time
from ultralytics import YOLO # YOLOv5, YOLOv8 등을 지원하는 최신 ultralytics 패키지 사용

# ===============================================
# 1. 로컬 경로 설정 (이전 코드에서 가져옴)
# ===============================================
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
train_annotations_dir = os.path.join(base_data_dir, 'train_annotations')

json_file_paths = []

if os.path.exists(train_annotations_dir):
    for root, dirs, files in os.walk(train_annotations_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                full_path = os.path.join(root, file_name)
                json_file_paths.append(full_path)

    if not json_file_paths:
        print("🚨 'train_annotations' 폴더에서 JSON 파일을 찾을 수 없습니다.")
        exit()

    print(f"'train_annotations'에서 발견된 총 JSON 파일 수: {len(json_file_paths)}개")
    first_json_path = json_file_paths[0]
    print(f"✅ 첫 번째 JSON 파일 경로 사용: {first_json_path}")
else:
    print(f"🚨 로컬 경로를 찾을 수 없습니다. {train_annotations_dir}")
    exit()

# ===============================================
# 2. JSON에서 이미지 경로 추정
# ===============================================
image_path = None
try:
    with open(first_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # JSON 구조에서 이미지 파일 이름 추출 (데이터셋에 맞게 'file_name'을 가정)
    image_filename = data.get('file_name') or data.get('filename') or os.path.basename(first_json_path).replace('_json', '').replace('.json', '.jpg')

    # 이미지 파일 경로 추정 (JSON과 동일한 디렉토리 또는 'train_images' 폴더를 가정)
    # 🚨🚨 이 부분은 데이터셋의 실제 이미지 저장 구조에 따라 수정이 필요할 수 있습니다. 🚨🚨

    # 1) JSON과 같은 폴더에 이미지가 있다고 가정
    image_dir = os.path.dirname(first_json_path)
    base_name = os.path.splitext(image_filename)[0]

    # jpg, png 순서로 시도
    image_path = os.path.join(image_dir, base_name + '.jpg')
    if not os.path.exists(image_path):
        image_path = os.path.join(image_dir, base_name + '.png')

    # 2) 'train_images' 폴더에 이미지가 있다고 가정 (추가적인 확인)
    if not os.path.exists(image_path):
        image_path = os.path.join(base_data_dir, 'train_images', image_filename)
        # 이미지 파일 이름에 확장자가 없는 경우 .jpg 또는 .png 추가 시도
        if not os.path.exists(image_path) and '.' not in image_filename:
             image_path = os.path.join(base_data_dir, 'train_images', base_name + '.jpg')

    if not image_path or not os.path.exists(image_path):
        print(f"🚨 이미지 파일을 찾을 수 없습니다. 추정된 경로: {image_path}")
        exit()

except Exception as e:
    print(f"🚨 JSON 파일 처리 중 오류 발생: {e}")
    exit()

# ===============================================
# 3. YOLOv5 추론 실행
# ===============================================
print(f"\n🔎 최종 이미지 파일 경로: {image_path}")
print("📦 YOLOv5s 모델 로드 및 추론 시작...")
start_time = time.time()

# 가장 빠르고 가벼운 사전 훈련된 YOLOv5s 모델 로드
model = YOLO('yolov5s.pt')

# 추론 실행 및 결과 자동 저장
# save=True: 탐지 결과를 이미지에 그려 'runs/detect/yolov5_quick_test' 폴더에 저장합니다.
results = model.predict(
    source=image_path,
    conf=0.25, # 최소 신뢰도
    iou=0.7,   # IOU 임계값
    save=True,
    name='yolov5_quick_test' # 결과 폴더 이름 설정
)

end_time = time.time()
print(f"\n✅ YOLOv5 추론 완료! 소요 시간: {end_time - start_time:.2f}초")

# ===============================================
# 4. 결과 출력 및 저장 위치 안내
# ===============================================
for r in results:
    detected_count = len(r.boxes)
    print(f"\n✨ 탐지된 객체 수: {detected_count}개")

    # 결과가 저장된 폴더 경로
    save_dir = r.save_dir
    print(f"💾 결과 이미지 저장 위치: {save_dir}")
    print("📢 결과를 확인하려면 PyCharm의 프로젝트 구조 또는 Finder에서 위 폴더를 열어보세요.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/bellboi/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
'train_annotations'에서 발견된 총 JSON 파일 수: 4526개
✅ 첫 번째 JSON 파일 경로 사용: /Users/bellboi/data/ai05-level1-project/train_annotations/K-003483-025367-027777-035206_json/K-027777/K-003483-025367-027777-035206_0_2_0_2_75_000_200.json
🚨 이미지 파일을 찾을 수 없습니다. 추정된 경로: /Users/bellboi/data/ai05-level1-project/train_images/K-003483-025367-027777-035206_0_2_0_2_75_000_200.jpg

🔎 최종 이미지 파일 경로: /Users/bellboi/data/ai05-level1-project/train_images/K-003483-025367-027777-035206_0_2_0_2_75_000_200.jpg
📦 YOLOv5s 모델 로드 및 추론 시작...
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and fea

FileNotFoundError: /Users/bellboi/data/ai05-level1-project/train_images/K-003483-025367-027777-035206_0_2_0_2_75_000_200.jpg does not exist

In [1]:
# @title JSON 경로에서 이미지 추출 및 YOLOv5 추론 - 수정된 이미지 경로 탐색 로직
# (앞부분 import 및 1. 로컬 경로 설정 코드는 동일하게 유지합니다.)

# ===============================================
# 2. JSON에서 이미지 경로 추정 (수정)
# ===============================================
print(f"\n✅ 첫 번째 JSON 파일 경로 사용: {first_json_path}")
image_path = None
try:
    # 1. JSON 파일 로드 및 이미지 파일 이름 추출
    with open(first_json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # JSON 구조에서 이미지 파일 이름 추출 ('file_name'을 가정)
    image_filename = data.get('file_name') or data.get('filename')

    # JSON 파일 이름 자체를 이미지 파일 이름으로 가정 (확장자만 다름)
    if not image_filename:
        # 'K-003483-025367-027777-035206_0_2_0_2_75_000_200.json' -> 'K-003483-025367-027777-035206_0_2_0_2_75_000_200'
        base_name = os.path.basename(first_json_path).replace('_json', '').replace('.json', '')
        image_filename = base_name

    # 2. 경로 탐색 시도 리스트
    # JSON 파일의 이름 (확장자 제외)
    base_name = os.path.splitext(image_filename)[0]

    # 탐색할 폴더 경로 리스트 (JSON 파일 경로를 기준으로 탐색)
    json_dir = os.path.dirname(first_json_path)
    json_parent_dir = os.path.dirname(json_dir) # '/.../K-003483-025367-027777-035206_json/K-027777'의 부모: '/.../K-003483-025367-027777-035206_json'

    search_dirs = [
        json_dir,         # 1. JSON 파일과 같은 폴더 (예: K-027777)
        json_parent_dir,  # 2. JSON 파일의 부모 폴더 (예: K-003483-...)
        os.path.join(base_data_dir, 'train_images') # 3. 기존에 시도했던 'train_images' (보조)
    ]

    # 3. 경로 탐색
    for directory in search_dirs:
        # .jpg, .png 확장자 시도
        for ext in ['.jpg', '.png']:
            current_path = os.path.join(directory, base_name + ext)
            if os.path.exists(current_path):
                image_path = current_path
                print(f"✅ 이미지 파일을 찾았습니다! 경로: {image_path}")
                break
        if image_path:
            break

    if not image_path:
        print(f"🚨 이미지 파일을 찾을 수 없습니다. 시도한 파일 이름: {base_name}.(jpg/png)")
        exit()

except Exception as e:
    print(f"🚨 JSON 파일 처리 중 오류 발생: {e}")
    exit()

# (3. YOLOv5 추론 실행 코드는 동일하게 유지합니다.)
# ===============================================
# 3. YOLOv5 추론 실행
# ===============================================
# ... (이하 코드는 동일)

NameError: name 'first_json_path' is not defined

In [2]:
# @title JSON 경로에서 이미지 추출 및 YOLOv5 추론 - 최종 버전 (경로 확정)
import os
import json
import time
from ultralytics import YOLO # Conda 환경에 설치된 ultralytics 사용

# ===============================================
# 1. 로컬 경로 설정 (이전 코드에서 가져옴)
# ===============================================
# Mac 로컬 환경의 데이터 기본 경로 설정
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
train_annotations_dir = os.path.join(base_data_dir, 'train_annotations')
train_images_dir = os.path.join(base_data_dir, 'train_images') # 이미지 폴더 경로 추가

json_file_paths = []
# ... (JSON 경로 수집 로직은 이전과 동일)
if os.path.exists(train_annotations_dir):
    start_time = time.time()
    for root, dirs, files in os.walk(train_annotations_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                json_file_paths.append(os.path.join(root, file_name))

    if not json_file_paths:
        print("🚨 'train_annotations' 폴더에서 JSON 파일을 찾을 수 없습니다.")
        exit()

    print(f"'train_annotations'에서 발견된 총 JSON 파일 수: {len(json_file_paths)}개")
    print(f"경로 수집 완료 시간: {time.time() - start_time:.2f}초")
    first_json_path = json_file_paths[0]
    print(f"✅ 첫 번째 JSON 파일 경로 사용: {first_json_path}")
else:
    print(f"🚨 로컬 경로를 찾을 수 없습니다. {train_annotations_dir}")
    exit()

# ===============================================
# 2. JSON에서 이미지 경로 추정 (구조 확정)
# ===============================================
image_path = None
try:
    # 1. JSON 파일 이름 추출 (확장자 제거)
    json_filename = os.path.basename(first_json_path)
    # 예: 'K-001900-016548-01811...200.json'

    # 2. 이미지 파일 이름 추정
    # JSON 파일 이름의 확장자(.json)를 제거하고 이미지 확장자(.png)를 붙입니다.
    # 제공된 이미지 파일은 모두 .png 확장자를 사용하고 있습니다.
    base_name_without_ext = os.path.splitext(json_filename)[0].replace('_json', '')
    image_filename = base_name_without_ext + '.png' # PNG 확정

    # 3. 최종 이미지 경로 생성 (train_images 폴더 사용)
    image_path = os.path.join(train_images_dir, image_filename)

    if not os.path.exists(image_path):
        # 만약 .png가 아니라 .jpg일 가능성도 있다면 (보조 확인)
        image_filename_jpg = base_name_without_ext + '.jpg'
        image_path = os.path.join(train_images_dir, image_filename_jpg)

        if not os.path.exists(image_path):
            print(f"🚨 이미지 파일을 찾을 수 없습니다.")
            print(f"시도한 경로 (.png): {os.path.join(train_images_dir, base_name_without_ext + '.png')}")
            print(f"시도한 경로 (.jpg): {os.path.join(train_images_dir, base_name_without_ext + '.jpg')}")
            exit()

    print(f"\n🔎 최종 이미지 파일 경로: {image_path}")

except Exception as e:
    print(f"🚨 JSON 파일 처리 중 오류 발생: {e}")
    exit()

# ===============================================
# 3. YOLOv5 추론 실행
# ===============================================
print("📦 YOLOv5s 모델 로드 및 추론 시작...")
start_time = time.time()

# 가장 빠르고 가벼운 사전 훈련된 YOLOv5s 모델 로드
model = YOLO('yolov5s.pt')

# 추론 실행 및 결과 자동 저장
# save=True: 탐지 결과를 이미지에 그려 'runs/detect/yolov5_quick_test' 폴더에 저장합니다.
results = model.predict(
    source=image_path,
    conf=0.25, # 최소 신뢰도
    iou=0.7,   # IOU 임계값
    save=True,
    name='yolov5_quick_test' # 결과 폴더 이름 설정
)

end_time = time.time()
print(f"\n✅ YOLOv5 추론 완료! 소요 시간: {end_time - start_time:.2f}초")

# ===============================================
# 4. 결과 출력 및 저장 위치 안내
# ===============================================
for r in results:
    detected_count = len(r.boxes)
    print(f"\n✨ 탐지된 객체 수: {detected_count}개")

    save_dir = r.save_dir
    print(f"💾 결과 이미지 저장 위치: {save_dir}")
    print("📢 결과를 확인하려면 PyCharm의 프로젝트 구조 또는 Finder에서 위 폴더를 열어보세요.")

'train_annotations'에서 발견된 총 JSON 파일 수: 4526개
경로 수집 완료 시간: 0.24초
✅ 첫 번째 JSON 파일 경로 사용: /Users/bellboi/data/ai05-level1-project/train_annotations/K-003483-025367-027777-035206_json/K-027777/K-003483-025367-027777-035206_0_2_0_2_75_000_200.json

🔎 최종 이미지 파일 경로: /Users/bellboi/data/ai05-level1-project/train_images/K-003483-025367-027777-035206_0_2_0_2_75_000_200.png
📦 YOLOv5s 모델 로드 및 추론 시작...
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


image 1/1 /Users/bellboi/data/ai05-level1-project/train_images/K-003483-025367-027777-035206_0_2_0_2_75_000_200.png: 640x512 1 sports ball, 1 remote, 141.3ms
Speed: 4.2ms preprocess, 141.3ms inference, 9.5ms postprocess per image at shape (1, 3, 640, 512)
Results saved to /Users/bellboi/code/codeitteam7/runs/detect/yolov5_quick_test

✅ YOLOv5 추론 완료

In [3]:
# @title JSON을 YOLO 포맷 TXT로 변환
# 주의: 이 코드는 JSON 구조에 따라 수정이 필요할 수 있습니다.

import os
import json
import time
from tqdm import tqdm # 진행 상황을 보기 쉽게 하기 위해 tqdm 사용

# 1. 경로 설정 (이전 코드에서 재사용)
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
train_annotations_dir = os.path.join(base_data_dir, 'train_annotations')
train_images_dir = os.path.join(base_data_dir, 'train_images')

# YOLO 라벨을 저장할 새로운 폴더 생성
yolo_labels_dir = os.path.join(base_data_dir, 'train_labels')
os.makedirs(yolo_labels_dir, exist_ok=True)
print(f"YOLO 라벨 저장 폴더 생성: {yolo_labels_dir}")

# 이전 단계에서 수집된 전체 JSON 파일 경로 리스트 (4526개)를 사용한다고 가정
# json_file_paths = ...

# ===============================================
# 2. 변환 함수 (COCO 포맷 기준)
# ===============================================
def json_to_yolo_txt(json_path, image_width, image_height):
    """
    JSON 파일 경로를 받아 YOLO 포맷의 TXT 문자열을 반환합니다.
    (x_min, y_min, w, h) -> (center_x, center_y, w, h) 정규화
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    yolo_lines = []

    # 이 부분은 데이터셋의 JSON 구조에 따라 반드시 수정해야 합니다.
    # 예시: COCO 스타일의 'annotations' 리스트를 가정
    annotations = data.get('annotations', [])

    for anno in annotations:
        try:
            # 1. 클래스 ID (데이터셋의 ID에 맞게 조정해야 합니다)
            class_id = anno['category_id']

            # 2. 바운딩 박스 (x_min, y_min, w, h 형태를 가정)
            bbox = anno['bbox'] # [x, y, w, h]
            x, y, w, h = bbox

            # 3. YOLO 포맷으로 변환 및 정규화
            center_x = (x + w / 2) / image_width
            center_y = (y + h / 2) / image_height
            norm_w = w / image_width
            norm_h = h / image_height

            # 4. YOLO 라인 생성: [class_id] [center_x] [center_y] [w] [h]
            line = f"{class_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}"
            yolo_lines.append(line)
        except KeyError as e:
            # 'annotations' 키 아래에 'category_id'나 'bbox'가 없을 경우
            print(f"경고: {json_path}에서 필수 키({e})를 찾을 수 없습니다. 스킵합니다.")
            continue

    return "\n".join(yolo_lines)

# ===============================================
# 3. 전체 JSON 파일 변환 실행
# ===============================================
print(f"\n총 {len(json_file_paths)}개의 JSON 파일을 YOLO TXT 포맷으로 변환 시작...")
start_time = time.time()

# 이미지 크기 정보 (임시): 현재 데이터셋의 이미지 크기를 모르므로, 첫 번째 이미지의 크기를 로드합니다.
# 실제로는 JSON 파일 안에 이미지 크기가 포함되어 있는 경우가 많습니다.
# 만약 YOLO 추론에서 사용된 640x512가 이미지 크기라면 이를 사용해야 합니다.
# 여기서는 실제 이미지 파일에서 크기를 얻어오겠습니다.
import cv2

def get_image_dimensions(image_filename):
    """train_images 폴더에서 파일 크기를 읽어옵니다."""
    base_name_without_ext = os.path.splitext(image_filename)[0].replace('_json', '').replace('.json', '')

    # PNG 파일을 먼저 시도
    img_path_png = os.path.join(train_images_dir, base_name_without_ext + '.png')
    if os.path.exists(img_path_png):
        img = cv2.imread(img_path_png)
        if img is not None:
             return img.shape[1], img.shape[0] # (width, height)

    # JPG 파일을 시도
    img_path_jpg = os.path.join(train_images_dir, base_name_without_ext + '.jpg')
    if os.path.exists(img_path_jpg):
        img = cv2.imread(img_path_jpg)
        if img is not None:
             return img.shape[1], img.shape[0] # (width, height)

    return None, None


for json_path in tqdm(json_file_paths, desc="변환 중"):
    # 1. 이미지 크기 얻기 (정규화를 위해 필요)
    json_filename = os.path.basename(json_path)
    img_w, img_h = get_image_dimensions(json_filename)

    if img_w is None or img_h is None:
        # 이미지 파일을 찾지 못하면 해당 JSON은 스킵
        continue

    # 2. YOLO TXT 내용 생성
    yolo_content = json_to_yolo_txt(json_path, img_w, img_h)

    # 3. TXT 파일로 저장
    # JSON 파일명과 동일하게 만들고 확장자만 .txt로 변경
    base_filename = os.path.splitext(os.path.basename(json_path))[0]
    txt_filename = base_filename.replace('_json', '') + '.txt'
    txt_path = os.path.join(yolo_labels_dir, txt_filename)

    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(yolo_content)

end_time = time.time()
print(f"\n✅ 전체 JSON 파일 변환 완료! 소요 시간: {end_time - start_time:.2f}초")
print(f"저장 위치 예시: {os.path.join(yolo_labels_dir, 'K-003483-025367-027777-035206_0_2_0_2_75_000_200.txt')}")

YOLO 라벨 저장 폴더 생성: /Users/bellboi/data/ai05-level1-project/train_labels

총 4526개의 JSON 파일을 YOLO TXT 포맷으로 변환 시작...


변환 중: 100%|██████████| 4526/4526 [01:35<00:00, 47.51it/s]


✅ 전체 JSON 파일 변환 완료! 소요 시간: 95.31초
저장 위치 예시: /Users/bellboi/data/ai05-level1-project/train_labels/K-003483-025367-027777-035206_0_2_0_2_75_000_200.txt


In [6]:
# @title YOLOv8 미니 학습 실행 (PyCharm)
import os
from ultralytics import YOLO

# 1. 경로 및 설정 파일 확인
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
data_yaml_path = os.path.join(os.getcwd(), 'ai05_pill_data.yaml') # 현재 스크립트가 있는 폴더에 있다고 가정

# train_images 폴더에 이미지를, train_labels 폴더에 TXT 라벨을 복사/이동했다고 가정합니다.
# 만약 라벨 폴더 이름을 train_labels 대신 labels로 사용하려면 data.yaml도 수정해야 합니다.

print(f"데이터 설정 파일 경로: {data_yaml_path}")

# 2. YOLOv8n 모델 로드 (가장 빠르고 가벼움)
# 'n'은 nano를 의미하며, 가장 가볍고 빠르게 학습/추론이 가능합니다.
model = YOLO('yolov8n.pt')

print("🚀 YOLOv8 모델로 알약 데이터 파인튜닝 시작 (최소 5 에폭)")

# 3. 학습 실행
# project/name: 결과가 저장될 폴더 경로를 지정합니다.
# data: 위에서 생성한 설정 파일 경로
# epochs: 학습 횟수 (빠른 확인을 위해 5회 설정)
# imgsz: 이미지 크기 (기본값인 640 사용)
# batch: 배치 크기 (GPU 메모리가 부족하면 4 또는 2로 줄이세요)
results = model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=8,
    project='yolo_quick_run',
    name='pill_detection_nano_v1'
)

print("\n✅ YOLOv8 미니 학습 완료!")
print(f"결과 저장 위치: {os.path.join(os.getcwd(), 'yolo_quick_run', 'pill_detection_nano_v1')}")
print("📢 'runs/detect/pill_detection_nano_v1' 폴더에서 결과 그래프와 학습된 모델을 확인할 수 있습니다.")

데이터 설정 파일 경로: /Users/bellboi/code/codeitteam7/ai05_pill_data.yaml
🚀 YOLOv8 모델로 알약 데이터 파인튜닝 시작 (최소 5 에폭)
Ultralytics 8.3.220 🚀 Python-3.9.6 torch-2.8.0 CPU (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_detection_nano_v13, nbs=64, nm

RuntimeError: No valid images found in /Users/bellboi/data/ai05-level1-project/labels.cache. Images with incorrectly formatted labels are ignored. See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [9]:
# 이전 변환 코드의 일부 (KeyError를 방지하기 위해 try/except로 묶여 있었음)
class_id = anno['category_id'] # JSON의 category_id를 그대로 사용

NameError: name 'anno' is not defined

In [8]:
# @title JSON을 YOLO 포맷 TXT로 변환 - 클래스 0으로 통일 (Test용)

# (이전 코드에서 json_to_yolo_txt 함수만 아래와 같이 수정합니다.)

def json_to_yolo_txt(json_path, image_width, image_height):
    # ... (파일 로드 부분은 동일)

    yolo_lines = []
    annotations = data.get('annotations', [])

    for anno in annotations:
        try:
            # 1. 클래스 ID: 임시로 무조건 0으로 설정하여 유효성 검사 통과 시도
            class_id = 0

            # 2. 바운딩 박스 (x_min, y_min, w, h 형태를 가정)
            bbox = anno['bbox'] # [x, y, w, h]
            x, y, w, h = bbox

            # 3. YOLO 포맷으로 변환 및 정규화
            center_x = (x + w / 2) / image_width
            center_y = (y + h / 2) / image_height
            norm_w = w / image_width
            norm_h = h / image_height

            # 4. 정규화된 값이 0~1 사이에 있는지 확인 (매우 중요)
            if not all(0 <= val <= 1.0 for val in [center_x, center_y, norm_w, norm_h]):
                 print(f"경고: {json_path}의 바운딩 박스 값이 0~1 범위를 벗어납니다. 스킵합니다.")
                 continue

            line = f"{class_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}"
            yolo_lines.append(line)
        except KeyError as e:
            # annotations, bbox, category_id 등의 키가 없는 경우
            continue # 해당 객체는 스킵하고 다음 객체로 진행

    return "\n".join(yolo_lines)

In [11]:
# @title JSON을 YOLO 포맷 TXT로 변환 - 클래스 0으로 통일 (Test용) - 통합 코드
import os
import json
import time
from tqdm import tqdm
import cv2 # 이미지 크기를 읽기 위해 필요

# ===============================================
# 1. 경로 설정 (이전 코드에서 재사용)
# ===============================================
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
train_annotations_dir = os.path.join(base_data_dir, 'train_annotations')
images_dir = os.path.join(base_data_dir, 'images') # 이름 변경된 'images' 폴더 사용
labels_dir = os.path.join(base_data_dir, 'labels') # 이름 변경된 'labels' 폴더 사용

os.makedirs(labels_dir, exist_ok=True)
print(f"YOLO 라벨 저장 폴더: {labels_dir}")

# JSON 파일 경로 수집 (이전 코드와 동일하게 JSON 경로를 먼저 채웁니다)
json_file_paths = []
if os.path.exists(train_annotations_dir):
    for root, dirs, files in os.walk(train_annotations_dir):
        for file_name in files:
            if file_name.endswith('.json'):
                json_file_paths.append(os.path.join(root, file_name))
    print(f"총 JSON 파일 수: {len(json_file_paths)}개")
else:
    print(f"🚨 JSON 경로를 찾을 수 없습니다: {train_annotations_dir}")
    exit()

# ===============================================
# 2. 이미지 크기 얻는 함수 (get_image_dimensions)
# ===============================================
def get_image_dimensions(json_filename):
    """images 폴더에서 파일 크기를 읽어옵니다."""
    # JSON 파일명에서 .json 확장자를 제거하고 이미지 파일명 추정
    base_name_without_ext = os.path.splitext(os.path.basename(json_filename))[0].replace('_json', '')

    # PNG 파일을 먼저 시도
    img_path_png = os.path.join(images_dir, base_name_without_ext + '.png')
    if os.path.exists(img_path_png):
        img = cv2.imread(img_path_png)
        if img is not None:
             return img.shape[1], img.shape[0] # (width, height)

    # JPG 파일을 시도
    img_path_jpg = os.path.join(images_dir, base_name_without_ext + '.jpg')
    if os.path.exists(img_path_jpg):
        img = cv2.imread(img_path_jpg)
        if img is not None:
             return img.shape[1], img.shape[0] # (width, height)

    return None, None

# ===============================================
# 3. 변환 함수 (클래스 0으로 통일, 바운딩 박스 범위 체크 추가)
# ===============================================
def json_to_yolo_txt(json_path, image_width, image_height):
    """JSON을 YOLO TXT로 변환 (클래스 ID는 0으로 고정)"""
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    yolo_lines = []
    # 데이터셋의 구조가 COCO와 유사하게 'annotations' 키를 사용한다고 가정
    annotations = data.get('annotations', [])

    for anno in annotations:
        try:
            # 1. 클래스 ID: 빠른 테스트를 위해 0으로 고정
            class_id = 0

            # 2. 바운딩 박스 (x_min, y_min, w, h 형태를 가정)
            bbox = anno.get('bbox') # [x, y, w, h]
            if not bbox or len(bbox) != 4: continue # 바운딩 박스 정보가 없으면 스킵

            x, y, w, h = bbox

            # 3. YOLO 포맷으로 변환 및 정규화
            center_x = (x + w / 2) / image_width
            center_y = (y + h / 2) / image_height
            norm_w = w / image_width
            norm_h = h / image_height

            # 4. 정규화된 값이 유효한지 확인 (0.0~1.0 범위를 벗어나면 YOLO에서 무시됨)
            if not all(0 <= val <= 1.0 for val in [center_x, center_y, norm_w, norm_h]):
                 # 값이 범위를 벗어난다면 해당 바운딩 박스는 버립니다.
                 continue

            line = f"{class_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}"
            yolo_lines.append(line)
        except Exception:
            # 데이터 구조가 다른 객체는 무시합니다.
            continue

    return "\n".join(yolo_lines)

## @title JSON을 YOLO 포맷 TXT로 변환 - 오타 수정 후 재실행

# (앞부분의 import, 함수 정의 등은 모두 동일하게 유지합니다.)

# ===============================================
# 4. 전체 JSON 파일 변환 실행 (메인 루프)
# ===============================================
print(f"\n총 {len(json_file_paths)}개의 JSON 파일을 YOLO TXT 포맷으로 변환 시작...")
start_time = time.time()

for json_path in tqdm(json_file_paths, desc="변환 중"):

    # 1. 이미지 크기 얻기
    json_filename = os.path.basename(json_path)
    img_w, img_h = get_image_dimensions(json_filename)

    if img_w is None or img_h is None:
        continue # 이미지 파일을 찾지 못하면 스킵

    # 2. YOLO TXT 내용 생성
    yolo_content = json_to_yolo_txt(json_path, img_w, img_h)

    # 3. TXT 파일로 저장
    base_filename = os.path.splitext(os.path.basename(json_path))[0] # 여기서 base_filename 정의

    # 🚨🚨 오타 수정 부분 🚨🚨
    txt_filename = base_filename.replace('_json', '') + '.txt' # 'base_' 대신 'base_filename' 사용

    txt_path = os.path.join(labels_dir, txt_filename)

    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(yolo_content)

end_time = time.time()
print(f"\n✅ 전체 JSON 파일 변환 완료! 소요 시간: {end_time - start_time:.2f}초")

YOLO 라벨 저장 폴더: /Users/bellboi/data/ai05-level1-project/labels
총 JSON 파일 수: 4526개

총 4526개의 JSON 파일을 YOLO TXT 포맷으로 변환 시작...


변환 중: 100%|██████████| 4526/4526 [01:28<00:00, 51.32it/s]


✅ 전체 JSON 파일 변환 완료! 소요 시간: 88.19초


In [13]:
# @title YOLOv8 미니 학습 실행 (최종)
import os
from ultralytics import YOLO
import torch # GPU 사용 가능 여부 확인용

# GPU 사용 가능 여부 확인 (Apple M2는 Metal Performance Shaders(MPS)를 사용합니다)
if torch.backends.mps.is_available():
    device = 'mps'
    print("✅ Apple M2 GPU (MPS) 사용 가능! 학습 속도가 빠를 수 있습니다.")
elif torch.cuda.is_available():
    device = 'cuda'
    print("✅ NVIDIA GPU (CUDA) 사용 가능!")
else:
    device = 'cpu'
    print("⚠️ CPU 모드로 학습을 시작합니다.")

# 1. 경로 및 설정 파일 확인
base_data_dir = '/Users/bellboi/data/ai05-level1-project'
# ai05_pill_data.yaml 파일이 현재 실행 폴더에 있다고 가정합니다.
data_yaml_path = os.path.join(os.getcwd(), 'ai05_pill_data.yaml')

print(f"\n데이터 설정 파일 경로: {data_yaml_path}")
print("🚀 YOLOv8 모델로 알약 데이터 파인튜닝 시작 (총 5 에폭)")

# 2. YOLOv8n 모델 로드
model = YOLO('yolov8n.pt')

# 3. 학습 실행
results = model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=8,
    project='yolo_quick_run',
    name='pill_detection_nano_final', # 이전 결과와 구분하기 위해 이름 변경
    device=device # 사용 가능한 장치로 지정
)

print("\n✅ YOLOv8 미니 학습 완료!")
print(f"결과 저장 위치: {os.path.join(os.getcwd(), 'yolo_quick_run', 'pill_detection_nano_final')}")

✅ Apple M2 GPU (MPS) 사용 가능! 학습 속도가 빠를 수 있습니다.

데이터 설정 파일 경로: /Users/bellboi/code/codeitteam7/ai05_pill_data.yaml
🚀 YOLOv8 모델로 알약 데이터 파인튜닝 시작 (총 5 에폭)
Ultralytics 8.3.220 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=F

/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/bellboi/code/codeitteam7/yolo_quick_run/pill_detection_nano_final2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
        1/5      2.29G     0.6865       2.16      0.957          2        640: 100% ━━━━━━━━━━━━ 187/187 0.7it/s 4:372.5ss
WARNING ⚠️ NMS time limit 2.800s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 1% ──────────── 1/94 0.0it/s 14.7s<1:15:47WARNING ⚠️ NMS time limit 2.800s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 2% ──────────── 2/94 0.1it

RuntimeError: Trying to create tensor with negative dimension -1: [1, -1, 5]

In [16]:
# @title YOLOv8 알약 데이터 학습 (Batch Size와 Workers 수 조정)
from ultralytics import YOLO
import torch
import os

# 1. 경로 설정 및 장치 확인
data_yaml_path = '/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml'

# MPS (Apple M2) 장치 사용
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f"✅ 사용 가능한 장치: {device.upper()}")

# 2. 모델 로드
# 모델 로드 경로: /Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/models/v8
model = YOLO('yolov8n.pt')

# 3. 학습 실행 (파라미터 조정)
# results = model.train(
#     data=data_yaml_path,
#     epochs=5,
#     imgsz=640,
#     # 🚨 Batch Size를 줄여 메모리 부하 감소 (8 -> 4)
#     batch=4,
#     project='yolo_quick_run',
#     name='pill_detection_nano_fix', # 이전 결과와 구분하기 위해 이름 변경
#     device=device,
#     # 🚨 Dataloader Workers 수를 줄여 안정성 확보 (8 -> 0)
#     workers=0,
#     # 🚨 NMS 경고 해결을 위해 평가 주기를 2 에폭마다 1회로 설정
#     # val_interval=2
# )

# @title 배치 크기를 1로 변경하여 다시 시도
model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=1, # 🚨 배치 크기를 1로 줄여 테스트
    project='yolo_quick_run',
    name='pill_detection_nano_batch1_test',
    device=device,
    workers=0,
)

print("\n✅ YOLOv8 학습 재개 완료! (Batch/Workers 조정)")
print(f"결과 저장 위치: {os.path.join(os.getcwd(), 'yolo_quick_run', 'pill_detection_nano_fix')}")

✅ 사용 가능한 장치: MPS
Ultralytics 8.3.220 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_detection_nano_batch1_test, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patienc

/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/bellboi/code/codeitteam7/yolo_quick_run/pill_detection_nano_batch1_test
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
: 0% ──────────── 0/1489  6.9s


RuntimeError: Trying to create tensor with negative dimension -1: [1, -1, 5]

In [17]:
# @title 레이블 파일 정리 후 재시도
# @title torch-2.8.0와 MPS 환경에서 발생한 RuntimeError는
# @title 대부분 빈 레이블 파일이나 Dataloader 불안정에서 기인합니다.
# @title 레이블 파일을 정리하고 다시 실행해 보세요.

# 코드의 시작엔 # @title 을 꼭 붙여줘.

import torch
import os
from ultralytics import YOLO

# MPS 장치 설정 (이미 활성화됨)
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
data_yaml_path = '/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml'

print(f"✅ 사용 가능한 장치: {device.upper()}")

# 1. 모델 로드
# 모델이 이미 로드되어 있다고 가정합니다. (yolov8n.pt)
model = YOLO('yolov8n.pt')

# 2. 학습 실행 (배치 크기 1 유지, workers 0 유지)
# 데이터셋 클리닝이 필요합니다.
results = model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=1, # 배치 크기를 1로 유지하여 문제 추적 용이
    project='yolo_quick_run',
    name='pill_detection_nano_batch1_fix_clean', # 이름 변경
    device=device,
    workers=0, # Dataloader 안정성 확보
    # cache='disk' # 데이터 로딩 안정화를 위해 캐시 사용 고려
)

✅ 사용 가능한 장치: MPS
Ultralytics 8.3.220 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_detection_nano_batch1_fix_clean, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/bellboi/code/codeitteam7/yolo_quick_run/pill_detection_nano_batch1_fix_clean
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
: 0% ──────────── 0/1489  1.0s


RuntimeError: Trying to create tensor with negative dimension -1: [1, -1, 5]

In [18]:
# @title 빈 레이블 파일 클리닝 스크립트

import os

# ⚠️ 사용자 경로에 맞게 이 경로를 반드시 수정하세요!
labels_dir = '/Users/bellboi/data/ai05-level1-project/labels'
# 만약 레이블이 images 폴더와 같은 레벨에 있다면, 경로는 '/Users/bellboi/data/ai05-level1-project/labels' 일 것입니다.
# 정확한 labels 폴더 경로를 확인하고 수정해 주세요.

if not os.path.isdir(labels_dir):
    print(f"❌ 오류: 레이블 디렉토리 '{labels_dir}'를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    print(f"🔍 '{labels_dir}'에서 빈 레이블 파일 검색 시작...")

    empty_files = []

    # 디렉토리 내 모든 .txt 파일을 검사
    for filename in os.listdir(labels_dir):
        if filename.endswith('.txt'):
            filepath = os.path.join(labels_dir, filename)

            # 파일 크기가 0이거나, 크기가 0이 아니더라도 내용이 공백만 있는 경우
            if os.path.getsize(filepath) == 0:
                is_empty = True
            else:
                with open(filepath, 'r') as f:
                    content = f.read().strip()
                    is_empty = not content

            if is_empty:
                empty_files.append(filepath)

    if empty_files:
        print(f"\n🚨 빈 레이블 파일 {len(empty_files)}개를 발견했습니다. 자동으로 제거합니다.")

        for filepath in empty_files:
            try:
                # 1. 빈 레이블 파일 제거
                os.remove(filepath)
                print(f"   🗑️ 제거된 레이블: {filepath}")

                # 2. 대응되는 이미지 파일도 제거 (선택 사항이지만 권장)
                # YOLO 포맷은 .txt 파일명과 .jpg 파일명이 같습니다. 확장자만 변경하여 제거
                base_name = os.path.splitext(filepath)[0]

                # 이미지 파일 경로 추정 (경로 구조에 따라 다를 수 있음. 예: labels 대신 images 폴더)
                # users/data/ai05-level1-project/labels/img1.txt -> users/data/ai05-level1-project/images/img1.jpg
                # 정확한 이미지 폴더 경로를 모르면 이 부분은 주석 처리하고 수동으로 제거

                # 예시: 'labels'를 'images'로 대체
                image_filepath_jpg = base_name.replace('labels', 'images') + '.jpg'
                image_filepath_png = base_name.replace('labels', 'images') + '.png'

                if os.path.exists(image_filepath_jpg):
                    os.remove(image_filepath_jpg)
                    print(f"   🗑️ 제거된 이미지: {image_filepath_jpg}")
                elif os.path.exists(image_filepath_png):
                    os.remove(image_filepath_png)
                    print(f"   🗑️ 제거된 이미지: {image_filepath_png}")

            except Exception as e:
                print(f"   ❌ 제거 실패 ({filepath}): {e}")

        print("\n✅ 빈 레이블 파일 및 대응되는 이미지(추정) 제거 완료. 학습을 다시 시도해 주세요.")
        print("💡 참고: YOLOv8이 레이블 캐시 파일을 재생성하도록 **`.cache` 파일을 삭제** 후 학습을 시작하세요.")

    else:
        print("\n✅ 빈 레이블 파일이 발견되지 않았습니다. 데이터셋은 깨끗합니다.")
        print("💡 그럼에도 오류가 발생한다면, Ultralytics/PyTorch/MPS **버전 문제**이거나 특정 이미지 데이터 자체의 로딩 문제입니다.")
        print("   다음 해결책(3. Dataloader 옵션 조정)을 시도해 보세요.")

🔍 '/Users/bellboi/data/ai05-level1-project/labels'에서 빈 레이블 파일 검색 시작...

🚨 빈 레이블 파일 1개를 발견했습니다. 자동으로 제거합니다.
   🗑️ 제거된 레이블: /Users/bellboi/data/ai05-level1-project/labels/K-003544-004543-012247-016551_0_2_0_2_70_000_200.txt
   🗑️ 제거된 이미지: /Users/bellboi/data/ai05-level1-project/images/K-003544-004543-012247-016551_0_2_0_2_70_000_200.png

✅ 빈 레이블 파일 및 대응되는 이미지(추정) 제거 완료. 학습을 다시 시도해 주세요.
💡 참고: YOLOv8이 레이블 캐시 파일을 재생성하도록 **`.cache` 파일을 삭제** 후 학습을 시작하세요.


In [19]:
 # @title 데이터셋 클리닝 완료 후 YOLOv8 학습 재개
# 코드의 시작엔 # @title 을 꼭 붙여줘.

import torch
from ultralytics import YOLO

# MPS 장치 설정 (이미 활성화됨)
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
data_yaml_path = '/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml'

print(f"✅ 사용 가능한 장치: {device.upper()}")

# 1. 모델 로드
model = YOLO('yolov8n.pt')

# 2. 학습 실행
# 'cache=ram' 또는 'cache=disk' 옵션을 사용하여 데이터 로딩 안정성을 높이는 것을 권장합니다.
# 데이터 클리닝을 했다는 가정 하에 batch=4로 늘려보겠습니다. (Batch=1에서 문제의 이미지를 제거했기를 기대)
results = model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=4, # 클린 후, 배치 사이즈를 다시 4로 시도
    project='yolo_quick_run',
    name='pill_detection_nano_batch4_clean_final',
    device=device,
    workers=0,
    cache='disk', # 데이터 로딩 안정화 옵션 추가
)

print("\n🎉 학습 재개 완료. 이제 Empty Label 문제가 해결되었기를 바랍니다.")

✅ 사용 가능한 장치: MPS
Ultralytics 8.3.220 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pill_detection_nano_batch4_clean_final, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, p

/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)


        1/5      2.53G     0.6673       2.22     0.9569          6        640: 84% ━━━━━━━━━━── 311/372 2.2it/s 4:58<27.8s


RuntimeError: Trying to create tensor with negative dimension -1: [4, -1, 5]

In [ ]:
# @title MPS 안정화 및 모자이크 증강 비활성화 후 YOLOv8 학습 재개
# 코드의 시작엔 # @title 을 꼭 붙여줘.

import torch
from ultralytics import YOLO

# MPS 장치 설정 (Apple M2)
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
data_yaml_path = '/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml'

print(f"✅ 사용 가능한 장치: {device.upper()}")
print(f"⚠️ MPS 환경의 불안정성 해결을 위해 'mosaic=0.0', 'workers=0'로 설정합니다.")

# 1. 모델 로드
model = YOLO('yolov8n.pt')

# 2. 학습 실행 (MPS 안정화 옵션 적용)
# batch=4는 유지합니다.
results = model.train(
    data=data_yaml_path,
    epochs=5,
    imgsz=640,
    batch=4,
    project='yolo_quick_run',
    name='pill_detection_nano_batch4_no_mosaic', # 이름 변경
    device=device,
    workers=0,
    cache='disk', # 데이터 로딩 안정화 유지
    # 💥 핵심 변경 사항: 모자이크 증강 비활성화
    mosaic=0.0,
)

print("\n🎉 모자이크 증강을 비활성화한 후 학습을 다시 시도했습니다. MPS 충돌 문제가 해결되었기를 바랍니다.")

✅ 사용 가능한 장치: MPS
⚠️ MPS 환경의 불안정성 해결을 위해 'mosaic=0.0', 'workers=0'로 설정합니다.
Ultralytics 8.3.220 🚀 Python-3.9.6 torch-2.8.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/bellboi/code/codeitteam7/ai05_pill_data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=pill_detection_nano_batch4_no_mosaic, nbs=64, nms=False, opset=Non

/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50508 (\N{HANGUL SYLLABLE AL}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)
/Users/bellboi/Library/Python/3.9/lib/python/site-packages/ultralytics/utils/plotting.py:627: UserWarning: Glyph 50557 (\N{HANGUL SYLLABLE YAG}) missing from font(s) DejaVu Sans.
  plt.savefig(fname, dpi=200)


        1/5      2.53G     0.5578      2.391       0.86          4        640: 100% ━━━━━━━━━━━━ 372/372 0.9it/s 7:16<0.9s
WARNING ⚠️ NMS time limit 2.400s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 1% ──────────── 1/186 0.0it/s 17.3s<2:58:17WARNING ⚠️ NMS time limit 2.400s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 1% ──────────── 2/186 0.1it/s 21.1s<33:28WARNING ⚠️ NMS time limit 2.400s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 2% ──────────── 3/186 0.1it/s 25.5s<23:07WARNING ⚠️ NMS time limit 2.400s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 2% ──────────── 4/186 0.2it/s 29.3s<17:43WARNING ⚠️ NMS time limit 2.400s exceeded
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 3% ──────────── 5/186 0.2it/s 32.7s<14:24WARNING ⚠️